In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",15)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels_20percent.pkl")
    kdd_train_2labels_y = pd.read_pickle("dataset/kdd_train_2labels_y_20percent.pkl")
    
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels_20percent.pkl")
    kdd_test_2labels_y = pd.read_pickle("dataset/kdd_test_2labels_y_20percent.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels_20percent.pkl")
    kdd_train_5labels_y = pd.read_pickle("dataset/kdd_train_5labels_y_20percent.pkl")
    
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels_20percent.pkl")
    kdd_test_5labels_y = pd.read_pickle("dataset/kdd_test_5labels_y_20percent.pkl")
    

In [5]:
import collections
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import NuSVC, SVC

names = ["Non - Linear SVM", "RBF SVM", "Decision Tree", "Random Forest", "AdaBoost","Multi Layer Perceptron"]
classifiers = [NuSVC(),
                SVC(gamma=2, C=1),
                DecisionTreeClassifier(max_depth=5),
                RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
                AdaBoostClassifier(),
                MLPClassifier(alpha=1),]
score = collections.namedtuple("score", ["name", "test_score"])
scores = []

for name, clf in zip(names, classifiers):
    print("Classifier: {}".format(name))
    
    clf.fit(dataset.kdd_train_2labels.iloc[:,:-2], 
        dataset.kdd_train_2labels_y)
    curr_score = clf.score(dataset.kdd_test_2labels.iloc[:,:-2], 
        dataset.kdd_test_2labels_y)
    scores.append(score(name, curr_score))
    print("Score: {}".format(curr_score))



Classifier: Non - Linear SVM
Score: 0.36455696202531646
Classifier: RBF SVM
Score: 0.2360337552742616
Classifier: Decision Tree
Score: 0.6070042194092827
Classifier: Random Forest
Score: 0.5723206751054852
Classifier: AdaBoost
Score: 0.589282700421941
Classifier: Multi Layer Perceptron
Score: 0.559324894514768


In [6]:
pd.DataFrame(scores)

,name,test_score
0,Non - Linear SVM,0.364557
1,RBF SVM,0.236034
2,Decision Tree,0.607004
3,Random Forest,0.572321
4,AdaBoost,0.589283
5,Multi Layer Perceptron,0.559325
